# 1 Carga de librerías

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats as stats
from scipy.stats import ttest_ind
from scipy.stats import levene

# 2. Carga de dataset

In [ ]:
# Uso de read_csv para leer los datos CSV, la "r" es para evitar los caracteres de escape
df = pd.read_csv(r'D:\Data analyst\datasets\datasets\games_full.csv')

In [ ]:
#Uso de info para ver datos relevantes de los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16450 entries, 0 to 16449
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16448 non-null  object 
 1   Platform         16450 non-null  object 
 2   Year_of_Release  16450 non-null  object 
 3   Genre            16448 non-null  object 
 4   NA_players       16450 non-null  float64
 5   EU_players       16450 non-null  float64
 6   JP_players       16450 non-null  float64
 7   Other_players    16450 non-null  float64
 8   Critic_Score     7983 non-null   float64
 9   User_Score       9840 non-null   object 
 10  Rating           9769 non-null   object 
dtypes: float64(5), object(6)
memory usage: 1.4+ MB


Comentarios:
- Nomenclatura inadecuada de las columnas, se procede a usar minúsculas y snake_case
- Solo presentamos 2 tipos de datos, objeto y flotantes, se indagará para hacer la transformación al dato correcto.
- Encontramos valores nulos, se indagará para tomar una buena decisión de su tratamiento.

In [ ]:
#Uso de head para ver rápidamente los datos
df.head(10)

,Name,Platform,Year_of_Release,Genre,NA_players,EU_players,JP_players,Other_players,Critic_Score,User_Score,Rating
0,Imagine: Makeup Artist,DS,2020-01-01,Simulation,0.27,0.00,0.00,0.02,NaN,tbd,E
1,Phantasy Star Online 2 Episode 4: Deluxe Package,PS4,2017-01-01,Role-Playing,0.00,0.00,0.04,0.00,NaN,NaN,NaN
2,Phantasy Star Online 2 Episode 4: Deluxe Package,PSV,2017-01-01,Role-Playing,0.00,0.00,0.01,0.00,NaN,NaN,NaN
3,Brothers Conflict: Precious Baby,PSV,2017-01-01,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
4,FIFA 17,PS4,2016-01-01,Sports,0.66,5.75,0.08,1.11,85.0,5,E
5,Pokemon Sun/Moon,3DS,2016-01-01,Role-Playing,2.98,1.45,2.26,0.45,NaN,NaN,NaN
6,Uncharted 4: A Thief's End,PS4,2016-01-01,Shooter,1.85,2.50,0.19,0.85,93.0,7.9,T
7,Call of Duty: Infinite Warfare,PS4,2016-01-01,Shooter,1.61,2.00,0.15,0.71,77.0,3.4,M
8,Battlefield 1,PS4,2016-01-01,Shooter,1.10,2.15,0.21,0.61,88.0,8.4,M
9,Tom Clancy's The Division,PS4,2016-01-01,Shooter,1.35,1.70,0.15,0.60,80.0,7,M


# 3 Preparación de datos

### Modificación de nombres

In [ ]:
#Ciclo for que itera sobre cada columna, aplicandole el método lower para volver a minúsculas
new_names = []
for old_names in df.columns:
    new_col_names = old_names.lower()
    new_names.append(new_col_names)
df.columns = new_names
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_players',
       'eu_players', 'jp_players', 'other_players', 'critic_score',
       'user_score', 'rating'],
      dtype='object')

### Evaluación y tratamiento de ausentes